In [3]:
import joblib
import pandas as pd
from better_profanity import profanity
import numpy as np

players = pd.read_csv("data/players.csv")

player_rounds = pd.read_csv("data/player_rounds.csv")

valid_ids = pd.read_csv("data/valid_ids.csv")

push_stats = pd.read_csv("data/push_stats.csv")

teams = pd.read_csv("data/teams.csv")

healspread = pd.read_csv("data/healspread.csv")

info = pd.read_csv("data/info.csv")

round_events = pd.read_csv('data/round_events.csv')

model_ready_data = joblib.load("data/pkls/model_ready_data_dict.pkl")
X = model_ready_data['X']

* Make PIM bindable

In [4]:
X

,scout_dapd_1,soldier_dapd_1,scout_dapm_1,soldier_dapm_1,scout_kpd_1,soldier_kpd_1,scout_offclass_pct_1,scout_kill_pct_1,soldier_kill_pct_1,scout_deaths_pct_1,...,medic_spy_assists_class_kdapd,medic_avg_time_before_healing,medic_avg_time_before_using,medic_avg_time_to_build,medic_avg_uber_length,medic_biggest_advantage_lost,medic_deaths_with_95_99_uber_rate,medic_deaths_within_20s_after_uber_rate,medic_drops_rate,medic_uberspm
0,0.321829,0.996185,0.311079,0.966405,0.209981,0.998832,0.000000,0.026984,0.994977,0.985587,...,0.000000,0.991765,0.250704,0.108131,0.539455,0.409080,0.000000,0.000000,0.000000,0.971005
1,0.061663,0.403138,0.201067,0.909140,0.071123,0.484351,0.763953,0.401176,0.993886,0.593643,...,0.000000,0.880157,0.217527,0.814464,0.961563,0.000000,0.000000,0.987565,0.000000,0.257054
2,0.970396,0.966168,0.971310,0.719791,0.987776,0.972088,0.000000,0.939724,0.260241,0.258336,...,0.423649,0.986745,0.080047,0.367866,0.430875,0.000000,0.000000,0.308254,0.664929,0.911302
3,0.004908,0.101378,0.035075,0.383409,0.023591,0.076183,0.312942,0.212483,0.453115,0.700452,...,0.423649,0.840157,0.023552,0.286552,0.971583,0.000000,0.000000,0.000000,0.000000,0.021824
4,0.340916,0.407304,0.190945,0.091171,0.268725,0.363905,0.354923,0.134491,0.127609,0.721954,...,0.423649,0.219294,0.390297,0.545895,0.878973,0.000000,0.000000,0.612363,0.000000,0.874279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6417,0.191298,0.123102,0.255139,0.038890,0.071123,0.484351,0.472276,0.165174,0.915615,0.364561,...,0.423649,0.237569,0.900548,0.017592,0.856819,0.000000,0.000000,0.748243,0.000000,0.045440
6418,0.040862,0.015884,0.399214,0.037099,0.023591,0.149486,0.000000,0.057628,0.255413,0.593643,...,0.423649,0.128000,0.082238,0.002189,0.829497,0.381087,0.000000,0.926473,0.000000,0.207015
6419,0.273450,0.117691,0.322563,0.368966,0.709514,0.363905,0.000000,0.186006,0.072079,0.852719,...,0.423649,0.331137,0.035290,0.033855,0.750587,0.000000,0.868209,0.415210,0.000000,0.485425
6420,0.783110,0.323756,0.233144,0.037099,0.458230,0.149486,0.302218,0.567479,0.358684,0.383453,...,0.000000,0.023373,0.983255,0.052776,0.156568,0.000000,0.000000,0.485943,0.000000,0.148870


In [ ]:
# read in
players_wide = model_ready_data['players_wide']
PIM = pd.read_csv("data/PIM_X.csv")
X = model_ready_data['X']


### GET PLAYER CLASS NAMES TO BE STEAMID + SCOUT1 / SOLDIER2 / DEMO
# Melt columns together to make a long dataset
cols = ['id','team'] + [col for col in players_wide.columns if "steamid" in col]
player_classes = players_wide[cols].copy()
player_class_long = player_classes.melt(id_vars=['id', 'team'], 
                  var_name='class', 
                  value_name='steamiplayers_wided')

# Make temp columns to transform 
player_class_long['class_clean'] = player_class_long['class'].str.split("_").str[0]
player_class_long['temp'] = player_class_long['class'].str.split("_").str[1]

# Fix up class name to be scout1 soldier2 instead of just scout soldier
class_list = []
for i,name in enumerate(player_class_long['class_clean']):
    if name == 'scout' or name == 'soldier':
        name += player_class_long.loc[i,'temp']
    class_list.append(name)
player_class_long['class'] = class_list
player_class_long.drop(['class_clean','temp'],axis = 1,inplace = True)


### MAKE PIM BINDABLE
PIM['id'] = X['id'].values


PIM_long = PIM.melt(
    id_vars=['id', 'winner'],
    var_name='class',
    value_name='PIM'
)

# Make pim_long
teams_pim = teams[['id','team','winner']].copy()
PIM_long = PIM_long.merge(teams_pim,on = ['id','winner'],how = "left")
PIM_long = PIM_long.merge(player_class_long,on = ['id','team','class'],how = 'left')

In [8]:
a.dropna()

,id,winner,class,PIM,team,steamiplayers_wided
0,2101331,1,demoman,7.31,Blue,[U:1:93362515]
1,2101331,0,demoman,3.88,Red,[U:1:86204943]
2,2101642,1,demoman,8.73,Blue,[U:1:99847536]
3,2101642,0,demoman,3.41,Red,[U:1:86204943]
4,2122702,1,demoman,6.41,Blue,[U:1:110979906]
...,...,...,...,...,...,...
38527,3886739,0,soldier2,0.70,Red,[U:1:89160056]
38528,3886745,0,soldier2,0.17,Blue,[U:1:132987261]
38529,3886745,1,soldier2,9.88,Red,[U:1:89195940]
38530,3886768,1,soldier2,5.38,Blue,[U:1:224635025]


In [45]:
a.dropna()

,id,winner,class,PIM,team,steamiplayers_wided
0,3781434,1,demoman,5.23,Blue,[U:1:113364548]
1,2730584,1,demoman,7.61,Blue,[U:1:160373133]
2,3778426,0,demoman,3.82,Blue,[U:1:339717315]
3,2826125,0,demoman,3.26,Red,[U:1:97691964]
4,2697489,0,demoman,3.33,Red,[U:1:133718175]
...,...,...,...,...,...,...
2311,2749117,1,soldier2,7.56,Red,[U:1:165063009]
2312,2939772,0,soldier2,2.14,Red,[U:1:83923591]
2313,3470120,0,soldier2,3.73,Blue,[U:1:287560752]
2314,2617947,1,soldier2,1.23,Blue,[U:1:837557388]


In [36]:
a[a['steamid'].isna()]

,id,winner,class,PIM,team,steamid
386,3781434,1,medic,2.24,Blue,NaN
387,2730584,1,medic,9.64,Blue,NaN
388,3778426,0,medic,3.50,Blue,NaN
389,2826125,0,medic,2.65,Red,NaN
390,2697489,0,medic,2.91,Red,NaN
...,...,...,...,...,...,...
767,2749117,1,medic,9.84,Red,NaN
768,2939772,0,medic,9.62,Red,NaN
769,3470120,0,medic,5.29,Blue,NaN
770,2617947,1,medic,7.06,Blue,NaN


In [30]:
PIM_long

,id,winner,class,PIM
0,3781434,1,demoman,5.23
1,2730584,1,demoman,7.61
2,3778426,0,demoman,3.82
3,2826125,0,demoman,3.26
4,2697489,0,demoman,3.33
...,...,...,...,...
2311,2749117,1,soldier2,7.56
2312,2939772,0,soldier2,2.14
2313,3470120,0,soldier2,3.73
2314,2617947,1,soldier2,1.23


In [17]:
player_class_long

,id,team,class,steamid
0,2101331,Blue,demoman,[U:1:93362515]
1,2101331,Red,demoman,[U:1:86204943]
2,2101642,Blue,demoman,[U:1:99847536]
3,2101642,Red,demoman,[U:1:86204943]
4,2122702,Blue,demoman,[U:1:110979906]
...,...,...,...,...
32105,3886739,Red,soldier2,[U:1:89160056]
32106,3886745,Blue,soldier2,[U:1:132987261]
32107,3886745,Red,soldier2,[U:1:89195940]
32108,3886768,Blue,soldier2,[U:1:224635025]


In [23]:
PIM_long['id'].nunique()

376

In [24]:
player_class_long

,id,team,class,steamid
0,2101331,Blue,demoman,[U:1:93362515]
1,2101331,Red,demoman,[U:1:86204943]
2,2101642,Blue,demoman,[U:1:99847536]
3,2101642,Red,demoman,[U:1:86204943]
4,2122702,Blue,demoman,[U:1:110979906]
...,...,...,...,...
32105,3886739,Red,soldier2,[U:1:89160056]
32106,3886745,Blue,soldier2,[U:1:132987261]
32107,3886745,Red,soldier2,[U:1:89195940]
32108,3886768,Blue,soldier2,[U:1:224635025]


In [22]:
PIM_long.sort_values(by = 'id')

,id,winner,class,PIM
1436,2122735,0,scout2,1.11
1050,2122735,0,scout1,3.49
278,2122735,0,demoman,1.58
2208,2122735,0,soldier2,6.27
1822,2122735,0,soldier1,3.15
...,...,...,...,...
1568,3886739,0,soldier1,2.85
1182,3886739,0,scout2,1.11
1954,3886739,0,soldier2,0.51
410,3886739,0,medic,0.12


In [4]:
X_eval['id']

5004    3781434
1204    2730584
4834    3778426
1929    2826125
735     2697489
         ...   
1461    2749117
2473    2939772
3644    3470120
598     2617947
4857    3778642
Name: id, Length: 386, dtype: int64

In [18]:
model_ready_data['players_wide']['id']

0       2101331
1       2101331
2       2101642
3       2101642
4       2122702
         ...   
6417    3886739
6418    3886745
6419    3886745
6420    3886768
6421    3886768
Name: id, Length: 6422, dtype: int64

In [ ]:
X_eval['id']

5004    0.779430
1204    0.187714
4834    0.752959
1929    0.300452
735     0.114528
          ...   
1461    0.227577
2473    0.385160
3644    0.567658
598     0.093351
4857    0.756384
Name: id, Length: 386, dtype: float64

,id,team,class,steamid
0,2101331,Blue,demoman,[U:1:93362515]
1,2101331,Red,demoman,[U:1:86204943]
2,2101642,Blue,demoman,[U:1:99847536]
3,2101642,Red,demoman,[U:1:86204943]
4,2122702,Blue,demoman,[U:1:110979906]
...,...,...,...,...
33095,3886745,Red,soldier2,[U:1:89195940]
33096,3886766,Blue,soldier2,[U:1:89195940]
33097,3886766,Red,soldier2,[U:1:117668615]
33098,3886768,Blue,soldier2,[U:1:224635025]


In [20]:
player_class_long.sort_values(by = 'id')

,id,team,class,steamid,class_clean,temp,class_fix
0,2101331,Blue,demoman_steamid,[U:1:93362515],demoman,steamid,demoman
1,2101331,Red,demoman_steamid,[U:1:86204943],demoman,steamid,demoman
6620,2101331,Blue,scout_1_steamid,[U:1:2638268],scout,1,scout1
6621,2101331,Red,scout_1_steamid,[U:1:152083482],scout,1,scout1
19860,2101331,Blue,soldier_1_steamid,[U:1:178192976],soldier,1,soldier1
...,...,...,...,...,...,...,...
6619,3886768,Red,demoman_steamid,[U:1:967251743],demoman,steamid,demoman
33098,3886768,Blue,soldier_2_steamid,[U:1:224635025],soldier,2,soldier2
19859,3886768,Red,scout_2_steamid,[U:1:236503569],scout,2,scout2
33099,3886768,Red,soldier_2_steamid,[U:1:87548232],soldier,2,soldier2


In [11]:
long_df

,id,team,class,steamid
0,2101331,Blue,demoman_steamid,[U:1:93362515]
1,2101331,Red,demoman_steamid,[U:1:86204943]
2,2101642,Blue,demoman_steamid,[U:1:99847536]
3,2101642,Red,demoman_steamid,[U:1:86204943]
4,2122702,Blue,demoman_steamid,[U:1:110979906]
...,...,...,...,...
33095,3886745,Red,soldier_2_steamid,[U:1:89195940]
33096,3886766,Blue,soldier_2_steamid,[U:1:89195940]
33097,3886766,Red,soldier_2_steamid,[U:1:117668615]
33098,3886768,Blue,soldier_2_steamid,[U:1:224635025]


In [ ]:
long_df['class'].str.split("_").str[0] + long_df['class'].str.split("_").str[1]

0          demomansteamid
1          demomansteamid
2          demomansteamid
3          demomansteamid
4          demomansteamid
                ...      
2396435          soldier2
2396436          soldier2
2396437          soldier2
2396438          soldier2
2396439          soldier2
Name: class, Length: 2396440, dtype: object

In [5]:
long_df['class'] = long_df['class'].str.split("_")[0]

ValueError: Length of values (2) does not match length of index (2396440)

In [60]:
model_ready_data['ids'].nunique()

3310

* Clean map name

In [63]:
# Find valid map names
info_correct = info[info['id'].isin(model_ready_data['ids'])].copy()
maps = info_correct['map'].str.lower().str.split("_")
map_counts = pd.Series(maps.str[1].value_counts())

valid_maps = map_counts[map_counts > 50]
valid_map_names = valid_maps.index

valid_map_names

Index(['process', 'snakewater', 'sunshine', 'gullywash', 'metalworks',
       'product', 'clearcut', 'villa', 'bagel'],
      dtype='object', name='map')

In [55]:
valid_map_names

Index(['process', 'snakewater', 'sunshine', 'gullywash', 'metalworks',
       'product', 'clearcut', 'villa', 'bagel'],
      dtype='object', name='map')

In [53]:
map_names

['process',
 'snakewater',
 'sunshine',
 'gullywash',
 'metalworks',
 'product',
 'clearcut',
 'villa',
 'bagel']

In [78]:
valid_info = info[info['id'].isin(valid_ids['id'])].copy()

map_names = list(valid_info['map'].str.lower().str.split("_").str[1].value_counts().index)

#valid_info['map'].str.lower().str.split("_").str[0].value_counts()

map_counts = []
for map in info['map'].str.lower().values:
    i = 0
    for map_name in map_names:
        if i > 0:
            continue
        if  type(map) == type(np.nan):
            map_counts.append(np.nan)
            i +=1
        elif map_name in map:
            map_counts.append(map_name)
            i += 1
        



# Define a function to find the first match
def find_match(text):
    if type(text) == type(np.nan):
        return np.nan
    for keyword in map_names:
        if keyword in text.lower():
            return keyword
    return np.nan  # if no match found

# Apply the function row by row
info['clean_map_name'] = info['map'].apply(find_match)
valid_info['clean_map_name'] = valid_info['map'].apply(find_match)

In [80]:
valid_info['clean_map_name'].value_counts()

clean_map_name
process       677
snakewater    560
sunshine      529
gullywash     434
metalworks    361
product       191
clearcut      181
villa         154
bagel         124
Name: count, dtype: int64

### Player Overview Tables ###

* Basic Stats Long

In [57]:
normal_cols = ['kills','deaths','assists','dmg','dmg_real','dt','dt_real','hr']
pct_cols = ['kill_pct','deaths_pct','assists_pct','dmg_pct','dmg_real_pct','dt_pct','dt_real_pct','hr_pct']

both_cols = ['id','team','steamid','name','primary_class']

normal_df = players[both_cols + normal_cols].copy()
pct_df = players[both_cols + pct_cols].copy()

pct_df.columns = normal_df.columns

normal_df['coltype'] = "Raw"
pct_df['coltype'] = "Pct of Team"


long_player = pd.concat([normal_df,pct_df])

long_player = long_player.fillna(0)

long_player.to_csv("data/long_player_stats.csv")

* Class KDA Long

In [28]:
class_cols = [col for col in players.columns if "class_kda" in col]

class_kill = [col for col in class_cols if "kills" in col and "kdapd" not in col]

class_deaths= [col for col in class_cols if "deaths" in col and "kdapd" not in col]

class_assists = [col for col in class_cols if "assists" in col and "kdapd" not in col]

class_kill_pd = [col for col in class_cols if "kills" in col and "kdapd" in col]

class_deaths_pd = [col for col in class_cols if "deaths" in col and "kdapd" in col]

class_assists_pd = [col for col in class_cols if "assists" in col and "kdapd" in col]

keep_cols = ['id','team','steamid','name','primary_class']



class_dict = {
    "Kills" : class_kill,
    "Deaths" : class_deaths,
    "Assists" : class_assists,
    "K/D" : class_kill_pd,
    "A/D" : class_assists_pd,
    "Death Rate" : class_deaths_pd
}

long_class = pd.DataFrame()
for key in class_dict.keys():
    col_type = key
    l = class_dict[key]
    df = players[keep_cols + l].copy()
    df.columns = keep_cols + [col.split("_")[0] for col in df.columns if col not in keep_cols] 
    df['col_type'] = col_type
    long_class = pd.concat([long_class,df])

long_class = long_class.fillna(0)

long_class.to_csv("data/class_kda_long.csv")

In [24]:
long_class[long_class['col_type'] == "Death Rate"]

,id,team,steamid,name,primary_class,demoman,heavyweapons,medic,pyro,scout,sniper,soldier,engineer,spy,col_type
0,3886768,Red,[U:1:113283780],loafe,medic,25.0%,0.0%,0.0%,0.0%,25.0%,0.0%,50.0%,0.0%,0.0%,Death Rate
1,3886768,Blue,[U:1:131704728],sushi,medic,50.0%,0.0%,0.0%,0.0%,0.0%,0.0%,50.0%,0.0%,0.0%,Death Rate
2,3886768,Blue,[U:1:136473263],Math,soldier,11.1%,0.0%,0.0%,0.0%,44.4%,0.0%,33.3%,5.6%,0.0%,Death Rate
3,3886768,Blue,[U:1:159413592],TIGER BALM,scout,22.2%,0.0%,0.0%,0.0%,66.7%,0.0%,11.1%,0.0%,0.0%,Death Rate
4,3886768,Red,[U:1:185989865],gabby,soldier,27.3%,0.0%,0.0%,0.0%,45.4%,0.0%,27.3%,0.0%,0.0%,Death Rate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53492,1764254,Red,[U:1:44119927],Kevin | Gas Station Soda - 99¢,pyro,0.0%,0.0%,0.0%,33.3%,0.0%,33.3%,0.0%,33.3%,0.0%,Death Rate
53493,1764254,Red,[U:1:58753664],Ambluj,spy,0.0%,0.0%,0.0%,33.3%,0.0%,0.0%,33.3%,16.7%,16.7%,Death Rate
53494,1764254,Blue,[U:1:78449395],Kevin | ProfessorParsnips,sniper,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,100.0%,Death Rate
53495,1764254,Blue,[U:1:82268202],Serru,scout,0.0%,0.0%,0.0%,25.0%,75.0%,0.0%,0.0%,0.0%,0.0%,Death Rate


* Attach player names to player_rounds

In [64]:
names = players[['id','steamid','name']]

player_rounds = player_rounds.merge(names, on = ['id','steamid']).drop("Unnamed: 0",axis= 1)

player_rounds.to_csv("data/player_rounds.csv")

* Quantile Columns

In [ ]:
# List out columns
quant_cols = [
    "kills",
    "assists",
    "dmg",
   'dapm',
   'kpd',
   'offclass_pct',
   'kill_pct',
   'deaths_pct',
   'dmg_pct',
   'dmg_real_pct',
   'cpc_pct',
   'ka_pct',
   'assists_pct',
   'dt_pct',
   'dt_real_pct',
   'hroi',
   'assistspd',
   'demoman_kills_class_kdapd',
   'scout_kills_class_kdapd',
   'soldier_kills_class_kdapd',
   'medic_kills_class_kdapd',
    'demoman_deaths_class_kdapd',
   'scout_deaths_class_kdapd',
   'soldier_deaths_class_kdapd',
    'medic_deaths_class_kdapd',
   'dtpm',
   'dt_realpm',
   'healpm',
   'medkits_hppm', 
   'hrpm', 
   'deathspm'

]

# Only grab valid ids to make quanitles on
sub_players = players[players['id'].isin(valid_ids['id'])].copy()
sub_players = sub_players[['id',"primary_class",'steamid']+ quant_cols]

# loop through classes and construct the quantile sets
ranked_df = pd.DataFrame()
for class_name in sub_players['primary_class'].unique():
    sub_class = sub_players[sub_players['primary_class'] == class_name].copy()
    
    binding_df = sub_class[['id',"primary_class",'steamid']].copy()
    sub_class.drop(['id',"primary_class",'steamid'],axis = 1,inplace= True)
    
    sub_class = sub_class.rank(pct = True)
    sub_class = pd.concat([binding_df,sub_class],axis = 1)
    
    ranked_df = pd.concat([ranked_df,sub_class])
    
# Rename columns
ranked_df.columns = ['id','primary_class','steamid'] + [col + '_quantile' for 
                                                        col in ranked_df.columns if
                                                        col not in ['id','steamid','primary_class']]

ranked_df.to_csv("data/players_quantile.csv")
